In [1]:
import numpy as np
import copy
import pickle
import wntr
import wntr.network.controls as controls
from itertools import combinations

In [2]:
def N_K_simulation(nk_pipe_list, wdn, conting_st, conting_et, seriest):
    # Get the nodes with demands
    junctions = wdn.junction_name_list
    nodes = []
    for j in junctions:
        j_object = wdn.get_node(j)
        base_demand = j_object.demand_timeseries_list[0].base_value
        if base_demand > 1e-8:
            nodes.append(j)
    
    # Perform simulation for normal operations
    wdno = copy.deepcopy(wdn)
    wdno.options.time.duration = 24 * 3600
    wdno.options.hydraulic.demand_model = 'PDD'
    simo = wntr.sim.EpanetSimulator(wdno)
    resultso = simo.run_sim() # by default run EPANET 2.2
    demandso = resultso.node['demand']
    # Calculate the expected total demand
    expected_total = {}
    for t in seriest:
        expected_total[t] = 0
        for n in nodes:
            expected_total[t] += demandso.loc[t*3600, n]
    
    # N-k contingency simulation
    functionality_loss = {}
    # The corresponding simulation results after turning off a set of pipe
    for nk_pipe in nk_pipe_list:
        # Initialization
        functionality_loss[nk_pipe] = {}
        for t in seriest:
            functionality_loss[nk_pipe][t] = 1
        
        # Make a copy of the original water distribution network
        wdnc = copy.deepcopy(wdn)
        ctrl1_list = []
        ctrl2_list = []
        for k in range(len(nk_pipe)):
            p_object = wdnc.get_link(nk_pipe[k])
            p_act1 = controls.ControlAction(p_object, 'status', 0)
            p_cond1 = controls.SimTimeCondition(wdnc, '=', str(conting_st) + ':00:00')
            ctrl1 = controls.Control(p_cond1, p_act1)
            ctrl1_list.append(ctrl1)
            p_act2 = controls.ControlAction(p_object, 'status', 1)
            p_cond2 = controls.SimTimeCondition(wdnc, '=', str(conting_et) + ':00:00')
            ctrl2 = controls.Control(p_cond2, p_act2)
            ctrl2_list.append(ctrl2)
        # Assign the controls on the network
        for m in range(len(ctrl1_list)):
            wdnc.add_control('Conting_start'+str(m), ctrl1_list[m])
            wdnc.add_control('Conting_end'+str(m), ctrl2_list[m])
        
        wdnc.options.time.duration = conting_et * 3600
        wdnc.options.hydraulic.demand_model = 'PDD'
        simc = wntr.sim.WNTRSimulator(wdnc)
        try:
            resultsc = simc.run_sim() # runs EPANET 2.2.0  version = 2.2
            actual_demands = resultsc.node['demand']
            for t in seriest:
                # some simulations may only have part of results 
                try:
                    actual_total = 0
                    for n in nodes:
                        actual_total += actual_demands.loc[t*3600, n]
                    loss = 1 - (actual_total / expected_total[t])
                    functionality_loss[nk_pipe][t] = loss
                except:
                    pass
        except:
            pass
    return functionality_loss

In [3]:
def generate_sample(n, k, pipe_list):
    samples = []
    index = range(len(pipe_list))
    for i in range(n):
        # Non-replaceable sampling
        select = np.random.choice(index, size = k, replace = False)
        samplei = []
        for ind in select:
            samplei.append(pipe_list[ind])
        samples.append(tuple(samplei))
    return samples       

In [4]:
# Load basic parameters
Wdn = wntr.network.WaterNetworkModel('LumbertonS.inp')
# The time of the contingency 
Conting_st = 0
# The end of the simulation
Simula_et = 24
Seriest = [int(t) for t in np.linspace(1, 23, 23)]
Pipe_list = Wdn.pipe_name_list

In [5]:
N = 10000
K = 3
Samples = generate_sample(N, K, Pipe_list)

In [6]:
Sample_simulation = N_K_simulation(Samples, Wdn, Conting_st, Simula_et, Seriest)

In [7]:
FSample_simulation = open('LumbertonS_N' + str(K) + '_sample2.pickle','wb')
pickle.dump(Sample_simulation, FSample_simulation)
FSample_simulation.close()